начало (левый верхний угол) 55.887292, 37.616198  
конец (правый нижний угол) 55.915078, 37.678408


Расчет параметров области

скрипт для скачивания и склейки

# Скрипт для джейсона с координатами и классами

Разметка полигонов на карте с точками в центрах

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
import os
from tqdm import tqdm

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
import os
from tqdm import tqdm

def process_large_image_tiled_with_centroids(model_path, image_path, output_path, tile_size=640, overlap=64, conf=0.2):
    """
    Обрабатывает большое изображение по частям и добавляет центроиды с разными маркерами для каждого класса
    """
    # Загружаем модель
    model = YOLO(model_path)
    
    # Загружаем исходное изображение
    original_image = cv2.imread(image_path)
    h, w = original_image.shape[:2]
    
    print(f"📐 Размер исходного изображения: {w}x{h}")
    print(f"🔲 Размер тайлов: {tile_size}x{tile_size}")
    print(f"🔄 Перекрытие: {overlap} пикселей")
    
    # Создаем пустое изображение для результатов
    result_image = original_image.copy()
    
    # Конфигурация маркеров для каждого класса
    marker_config = {
        'building': {'shape': 'square', 'color': (0, 0, 255), 'size': 10},      # красный квадрат
        'field': {'shape': 'diamond', 'color': (0, 255, 0), 'size': 12},        # зеленый ромб
        'forest': {'shape': 'circle', 'color': (0, 100, 0), 'size': 8},         # темно-зеленый круг
        'lake': {'shape': 'triangle_up', 'color': (255, 0, 0), 'size': 12},     # синий треугольник вверх
        'road': {'shape': 'triangle_down', 'color': (128, 128, 128), 'size': 10}, # серый треугольник вниз
        'zrail': {'shape': 'cross', 'color': (0, 165, 255), 'size': 15}         # оранжевый крестик
    }
    
    # Для подсчета статистики
    class_counts = {}
    total_centroids = 0
    
    # Вычисляем шаги для тайлов
    x_steps = (w - overlap) // (tile_size - overlap)
    y_steps = (h - overlap) // (tile_size - overlap)
    
    total_tiles = x_steps * y_steps
    print(f"🧩 Количество тайлов: {total_tiles}")
    
    processed_tiles = 0
    
    # Обрабатываем каждый тайл
    for y in tqdm(range(0, h, tile_size - overlap), desc="Обработка тайлов"):
        for x in range(0, w, tile_size - overlap):
            # Вычисляем координаты тайла с учетом границ
            x1 = x
            y1 = y
            x2 = min(x + tile_size, w)
            y2 = min(y + tile_size, h)
            
            # Если тайл слишком маленький, пропускаем
            if (x2 - x1) < tile_size // 2 or (y2 - y1) < tile_size // 2:
                continue
            
            # Вырезаем тайл
            tile = original_image[y1:y2, x1:x2]
            
            # Обрабатываем тайл моделью
            results = model(tile, conf=conf, verbose=False)
            
            # Обрабатываем результаты для этого тайла
            for r in results:
                if r.masks is not None and len(r.masks) > 0:
                    for i, mask in enumerate(r.masks.data):
                        # Получаем класс объекта
                        class_id = int(r.boxes.cls[i])
                        class_name = model.names[class_id]
                        confidence = float(r.boxes.conf[i])
                        
                        # Конвертируем маску в numpy array
                        mask_np = mask.cpu().numpy()
                        
                        # Масштабируем маску к размеру тайла
                        mask_resized = cv2.resize(mask_np, (x2-x1, y2-y1))
                        mask_binary = (mask_resized > 0.5).astype(np.uint8) * 255
                        
                        # Вычисляем центроид маски
                        centroid = calculate_centroid(mask_binary)
                        
                        if centroid:
                            # Преобразуем координаты центроида в глобальные
                            global_centroid = (x1 + centroid[0], y1 + centroid[1])
                            
                            # Получаем настройки маркера для этого класса
                            config = marker_config.get(class_name, {
                                'shape': 'circle', 
                                'color': (255, 255, 0), 
                                'size': 8
                            })
                            
                            # Рисуем маркер на итоговом изображении
                            draw_centroid_marker(
                                result_image, 
                                global_centroid, 
                                config['shape'], 
                                config['color'], 
                                config['size']
                            )
                            
                            # Добавляем подпись с классом и уверенностью
                            label = f"{class_name} ({confidence:.2f})"
                            cv2.putText(result_image, label, 
                                      (global_centroid[0] + 15, global_centroid[1]), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                                      config['color'], 2)
                            
                            # Обновляем статистику
                            class_counts[class_name] = class_counts.get(class_name, 0) + 1
                            total_centroids += 1
            
            # Также рисуем стандартные полигоны (опционально)
            for r in results:
                if hasattr(r, 'plot') and r.boxes is not None:
                    plotted_tile = r.plot()
                    result_image[y1:y2, x1:x2] = plotted_tile
            
            processed_tiles += 1
    
    # Сохраняем результат
    cv2.imwrite(output_path, result_image)
    print(f"💾 Результат сохранен: {output_path}")
    print(f"✅ Обработано тайлов: {processed_tiles}/{total_tiles}")
    
    # Выводим статистику
    print(f"🎯 Всего обнаружено центроидов: {total_centroids}")
    print("📊 Статистика по классам:")
    for class_name, count in class_counts.items():
        marker_shape = marker_config.get(class_name, {}).get('shape', 'circle')
        print(f"  - {class_name}: {count} объектов (маркер: {marker_shape})")
    
    return result_image

def calculate_centroid(mask):
    """
    Вычисляет центроид бинарной маски
    """
    try:
        # Находим моменты маски
        moments = cv2.moments(mask)
        
        if moments["m00"] != 0:
            centroid_x = int(moments["m10"] / moments["m00"])
            centroid_y = int(moments["m01"] / moments["m00"])
            return (centroid_x, centroid_y)
        else:
            # Если моменты нулевые, используем bounding rect
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                x, y, w, h = cv2.boundingRect(contours[0])
                return (x + w//2, y + h//2)
            else:
                return None
    except Exception as e:
        print(f"Ошибка при вычислении центроида: {e}")
        return None

def draw_centroid_marker(image, center, shape, color, size):
    """
    Рисует маркер в указанной точке
    """
    x, y = int(center[0]), int(center[1])
    half_size = size // 2
    
    try:
        if shape == 'circle':
            cv2.circle(image, (x, y), size, color, -1)
            cv2.circle(image, (x, y), size, (255, 255, 255), 2)  # белая обводка
            
        elif shape == 'square':
            cv2.rectangle(image, (x-half_size, y-half_size), 
                        (x+half_size, y+half_size), color, -1)
            cv2.rectangle(image, (x-half_size, y-half_size), 
                        (x+half_size, y+half_size), (255, 255, 255), 2)
            
        elif shape == 'diamond':
            points = np.array([
                [x, y-half_size],
                [x+half_size, y],
                [x, y+half_size],
                [x-half_size, y]
            ], dtype=np.int32)
            cv2.fillPoly(image, [points], color)
            cv2.polylines(image, [points], True, (255, 255, 255), 2)
            
        elif shape == 'triangle_up':
            points = np.array([
                [x, y-half_size],
                [x-half_size, y+half_size],
                [x+half_size, y+half_size]
            ], dtype=np.int32)
            cv2.fillPoly(image, [points], color)
            cv2.polylines(image, [points], True, (255, 255, 255), 2)
            
        elif shape == 'triangle_down':
            points = np.array([
                [x, y+half_size],
                [x-half_size, y-half_size],
                [x+half_size, y-half_size]
            ], dtype=np.int32)
            cv2.fillPoly(image, [points], color)
            cv2.polylines(image, [points], True, (255, 255, 255), 2)
            
        elif shape == 'cross':
            cv2.line(image, (x-half_size, y-half_size), 
                    (x+half_size, y+half_size), color, 3)
            cv2.line(image, (x-half_size, y+half_size), 
                    (x+half_size, y-half_size), color, 3)
            cv2.circle(image, (x, y), 2, (255, 255, 255), -1)
            
    except Exception as e:
        print(f"Ошибка при рисовании маркера: {e}")

# Альтернативная версия - только центроиды без полигонов
def process_large_image_centroids_only(model_path, image_path, output_path, tile_size=640, overlap=64, conf=0.2):
    """
    Только центроиды без отрисовки полигонов (более чистое изображение)
    """
    model = YOLO(model_path)
    original_image = cv2.imread(image_path)
    h, w = original_image.shape[:2]
    
    print(f"📐 Размер изображения: {w}x{h}")
    
    result_image = original_image.copy()
    
    # Конфигурация маркеров
    marker_config = {
        'building': {'shape': 'square', 'color': (0, 0, 255), 'size': 8},
        'field': {'shape': 'diamond', 'color': (0, 255, 0), 'size': 10},
        'forest': {'shape': 'circle', 'color': (0, 100, 0), 'size': 6},
        'lake': {'shape': 'triangle_up', 'color': (255, 0, 0), 'size': 10},
        'road': {'shape': 'triangle_down', 'color': (128, 128, 128), 'size': 8},
        'zrail': {'shape': 'cross', 'color': (0, 165, 255), 'size': 12}
    }
    
    class_counts = {}
    total_centroids = 0
    
    for y in tqdm(range(0, h, tile_size - overlap), desc="Обработка тайлов"):
        for x in range(0, w, tile_size - overlap):
            x1, y1 = x, y
            x2, y2 = min(x + tile_size, w), min(y + tile_size, h)
            
            if (x2 - x1) < tile_size // 2 or (y2 - y1) < tile_size // 2:
                continue
            
            tile = original_image[y1:y2, x1:x2]
            results = model(tile, conf=conf, verbose=False)
            
            for r in results:
                if r.masks is not None:
                    for i, mask in enumerate(r.masks.data):
                        class_id = int(r.boxes.cls[i])
                        class_name = model.names[class_id]
                        confidence = float(r.boxes.conf[i])
                        
                        mask_np = mask.cpu().numpy()
                        mask_resized = cv2.resize(mask_np, (x2-x1, y2-y1))
                        mask_binary = (mask_resized > 0.5).astype(np.uint8) * 255
                        
                        centroid = calculate_centroid(mask_binary)
                        
                        if centroid:
                            global_centroid = (x1 + centroid[0], y1 + centroid[1])
                            
                            config = marker_config.get(class_name, {
                                'shape': 'circle', 'color': (255, 255, 0), 'size': 6
                            })
                            
                            draw_centroid_marker(
                                result_image, 
                                global_centroid, 
                                config['shape'], 
                                config['color'], 
                                config['size']
                            )
                            
                            # Упрощенная подпись (только класс)
                            cv2.putText(result_image, class_name, 
                                      (global_centroid[0] + 12, global_centroid[1]), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.4, 
                                      config['color'], 1)
                            
                            class_counts[class_name] = class_counts.get(class_name, 0) + 1
                            total_centroids += 1
    
    cv2.imwrite(output_path, result_image)
    print(f"💾 Результат сохранен: {output_path}")
    print(f"🎯 Всего центроидов: {total_centroids}")
    
    return result_image

# Использование
if __name__ == "__main__":
    model_path = 'runs/segment/yolov8n_gpu_simple_1/weights/best.pt'
    image_path = 'output_img/my_5km_map.jpg'
    output_path = 'output_yolo_img/my_5km_map_with_centroids.jpg'
    
    print("🚀 Запуск обработки с центроидами...")
    start_time = time.time()
    
    # Вариант 1: С полигонами и центроидами
    result = process_large_image_tiled_with_centroids(
        model_path, 
        image_path, 
        output_path,
        tile_size=640,
        overlap=64,
        conf=0.2
    )
    
    # Вариант 2: Только центроиды (более чистое изображение)
    # result = process_large_image_centroids_only(
    #     model_path,
    #     image_path,
    #     'my_5km_map_centroids_only.jpg',
    #     tile_size=640,
    #     overlap=64,
    #     conf=0.2
    # )
    
    end_time = time.time()
    print(f"⏱️ Общее время обработки: {end_time - start_time:.2f} секунд")

🚀 Запуск обработки с центроидами...
📐 Размер исходного изображения: 11776x9472
🔲 Размер тайлов: 640x640
🔄 Перекрытие: 64 пикселей
🧩 Количество тайлов: 320


Обработка тайлов: 100%|██████████| 17/17 [01:02<00:00,  3.66s/it]


💾 Результат сохранен: my_5km_map_with_centroids.jpg
✅ Обработано тайлов: 320/320
🎯 Всего обнаружено центроидов: 1664
📊 Статистика по классам:
  - building: 1521 объектов (маркер: square)
  - lake: 85 объектов (маркер: triangle_up)
  - forest: 41 объектов (маркер: circle)
  - field: 17 объектов (маркер: diamond)
⏱️ Общее время обработки: 64.52 секунд


In [13]:
# Вариант 2: Только центроиды (более чистое изображение)
result = process_large_image_centroids_only(
    model_path,
    image_path,
    'my_5km_map_centroids_only.jpg',
    tile_size=640,
    overlap=64,
    conf=0.2
)

📐 Размер изображения: 11776x9472


Обработка тайлов: 100%|██████████| 17/17 [00:51<00:00,  3.01s/it]


💾 Результат сохранен: my_5km_map_centroids_only.jpg
🎯 Всего центроидов: 1664


In [14]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
import os
import json
from tqdm import tqdm
from datetime import datetime

def process_large_image_tiled_with_centroids_and_geojson(model_path, image_path, output_path, 
                                                         tile_size=640, overlap=64, conf=0.2,
                                                         top_left_gps=(55.887292, 37.616198),
                                                         bottom_right_gps=(55.915078, 37.678408)):
    """
    Обрабатывает большое изображение по частям, добавляет центроиды с маркерами и сохраняет координаты в JSON
    """
    # Загружаем модель
    model = YOLO(model_path)
    
    # Загружаем исходное изображение
    original_image = cv2.imread(image_path)
    h, w = original_image.shape[:2]
    
    print(f"📐 Размер исходного изображения: {w}x{h}")
    print(f"🔲 Размер тайлов: {tile_size}x{tile_size}")
    print(f"🔄 Перекрытие: {overlap} пикселей")
    print(f"🗺️  Географические координаты:")
    print(f"   Верхний левый: {top_left_gps}")
    print(f"   Нижний правый: {bottom_right_gps}")
    
    # Создаем пустое изображение для результатов
    result_image = original_image.copy()
    
    # Конфигурация маркеров для каждого класса
    marker_config = {
        'building': {'shape': 'square', 'color': (0, 0, 255), 'size': 10},
        'field': {'shape': 'diamond', 'color': (0, 255, 0), 'size': 12},
        'forest': {'shape': 'circle', 'color': (0, 100, 0), 'size': 8},
        'lake': {'shape': 'triangle_up', 'color': (255, 0, 0), 'size': 12},
        'road': {'shape': 'triangle_down', 'color': (128, 128, 128), 'size': 10},
        'zrail': {'shape': 'cross', 'color': (0, 165, 255), 'size': 15}
    }
    
    # Для хранения данных о центроидах
    centroids_data = {
        'metadata': {
            'image_path': image_path,
            'image_size': {'width': w, 'height': h},
            'gps_bounds': {
                'top_left': top_left_gps,
                'bottom_right': bottom_right_gps
            },
            'processing_date': datetime.now().isoformat(),
            'model_used': model_path
        },
        'objects': []
    }
    
    # Вычисляем шаги для тайлов
    x_steps = (w - overlap) // (tile_size - overlap)
    y_steps = (h - overlap) // (tile_size - overlap)
    
    total_tiles = x_steps * y_steps
    print(f"🧩 Количество тайлов: {total_tiles}")
    
    processed_tiles = 0
    
    # Обрабатываем каждый тайл
    for y in tqdm(range(0, h, tile_size - overlap), desc="Обработка тайлов"):
        for x in range(0, w, tile_size - overlap):
            # Вычисляем координаты тайла с учетом границ
            x1 = x
            y1 = y
            x2 = min(x + tile_size, w)
            y2 = min(y + tile_size, h)
            
            # Если тайл слишком маленький, пропускаем
            if (x2 - x1) < tile_size // 2 or (y2 - y1) < tile_size // 2:
                continue
            
            # Вырезаем тайл
            tile = original_image[y1:y2, x1:x2]
            
            # Обрабатываем тайл моделью
            results = model(tile, conf=conf, verbose=False)
            
            # Обрабатываем результаты для этого тайла
            for r in results:
                if r.masks is not None and len(r.masks) > 0:
                    for i, mask in enumerate(r.masks.data):
                        # Получаем класс объекта
                        class_id = int(r.boxes.cls[i])
                        class_name = model.names[class_id]
                        confidence = float(r.boxes.conf[i])
                        
                        # Конвертируем маску в numpy array
                        mask_np = mask.cpu().numpy()
                        
                        # Масштабируем маску к размеру тайла
                        mask_resized = cv2.resize(mask_np, (x2-x1, y2-y1))
                        mask_binary = (mask_resized > 0.5).astype(np.uint8) * 255
                        
                        # Вычисляем центроид маски
                        centroid = calculate_centroid(mask_binary)
                        
                        if centroid:
                            # Преобразуем координаты центроида в глобальные (пиксели)
                            global_centroid_px = (x1 + centroid[0], y1 + centroid[1])
                            
                            # Конвертируем пиксельные координаты в GPS
                            gps_coords = pixel_to_gps(global_centroid_px, w, h, top_left_gps, bottom_right_gps)
                            
                            # Получаем настройки маркера для этого класса
                            config = marker_config.get(class_name, {
                                'shape': 'circle', 
                                'color': (255, 255, 0), 
                                'size': 8
                            })
                            
                            # Рисуем маркер на итоговом изображении
                            draw_centroid_marker(
                                result_image, 
                                global_centroid_px, 
                                config['shape'], 
                                config['color'], 
                                config['size']
                            )
                            
                            # Добавляем подпись с классом и уверенностью
                            label = f"{class_name} ({confidence:.2f})"
                            cv2.putText(result_image, label, 
                                      (global_centroid_px[0] + 15, global_centroid_px[1]), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                                      config['color'], 2)
                            
                            # Сохраняем данные о центроиде (только class_id, confidence и gps_coordinates)
                            centroid_info = {
                                'class_id': class_id,
                                'confidence': confidence,
                                'gps_coordinates': {
                                    'latitude': gps_coords[0],
                                    'longitude': gps_coords[1]
                                }
                            }
                            centroids_data['objects'].append(centroid_info)
            
            # Также рисуем стандартные полигоны (опционально)
            for r in results:
                if hasattr(r, 'plot') and r.boxes is not None:
                    plotted_tile = r.plot()
                    result_image[y1:y2, x1:x2] = plotted_tile
            
            processed_tiles += 1
    
    # Сохраняем изображение с результатами
    cv2.imwrite(output_path, result_image)
    print(f"💾 Изображение с центроидами сохранено: {output_path}")
    
    # Сохраняем данные в JSON
    json_output_path = output_path.replace('.jpg', '_centroids.json').replace('.png', '_centroids.json')
    save_centroids_to_json(centroids_data, json_output_path)
    
    # Выводим статистику
    print(f"✅ Обработано тайлов: {processed_tiles}/{total_tiles}")
    print(f"🎯 Всего обнаружено центроидов: {len(centroids_data['objects'])}")
    
    # Статистика по классам
    class_counts = {}
    for obj in centroids_data['objects']:
        class_id = obj['class_id']
        class_name = model.names[class_id]
        class_counts[class_name] = class_counts.get(class_name, 0) + 1
    
    print("📊 Статистика по классам:")
    for class_name, count in class_counts.items():
        marker_shape = marker_config.get(class_name, {}).get('shape', 'circle')
        print(f"  - {class_name}: {count} объектов (маркер: {marker_shape})")
    
    return result_image, centroids_data

def pixel_to_gps(pixel_coords, image_width, image_height, top_left_gps, bottom_right_gps):
    """
    Конвертирует пиксельные координаты в GPS координаты
    
    Args:
        pixel_coords: (x, y) координаты в пикселях
        image_width, image_height: размеры изображения
        top_left_gps: (lat, lon) верхнего левого угла
        bottom_right_gps: (lat, lon) нижнего правого угла
    
    Returns:
        tuple: (latitude, longitude)
    """
    x_px, y_px = pixel_coords
    
    # Вычисляем соотношения
    lat_range = top_left_gps[0] - bottom_right_gps[0]  # широта уменьшается сверху вниз
    lon_range = bottom_right_gps[1] - top_left_gps[1]  # долгота увеличивается слева направо
    
    # Нормализуем пиксельные координаты (0-1)
    x_norm = x_px / image_width
    y_norm = y_px / image_height
    
    # Вычисляем GPS координаты
    latitude = top_left_gps[0] - (y_norm * lat_range)
    longitude = top_left_gps[1] + (x_norm * lon_range)
    
    return (round(latitude, 6), round(longitude, 6))

def calculate_centroid(mask):
    """
    Вычисляет центроид бинарной маски
    """
    try:
        # Находим моменты маски
        moments = cv2.moments(mask)
        
        if moments["m00"] != 0:
            centroid_x = int(moments["m10"] / moments["m00"])
            centroid_y = int(moments["m01"] / moments["m00"])
            return (centroid_x, centroid_y)
        else:
            # Если моменты нулевые, используем bounding rect
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                x, y, w, h = cv2.boundingRect(contours[0])
                return (x + w//2, y + h//2)
            else:
                return None
    except Exception as e:
        print(f"Ошибка при вычислении центроида: {e}")
        return None

def draw_centroid_marker(image, center, shape, color, size):
    """
    Рисует маркер в указанной точке
    """
    x, y = int(center[0]), int(center[1])
    half_size = size // 2
    
    try:
        if shape == 'circle':
            cv2.circle(image, (x, y), size, color, -1)
            cv2.circle(image, (x, y), size, (255, 255, 255), 2)
            
        elif shape == 'square':
            cv2.rectangle(image, (x-half_size, y-half_size), 
                        (x+half_size, y+half_size), color, -1)
            cv2.rectangle(image, (x-half_size, y-half_size), 
                        (x+half_size, y+half_size), (255, 255, 255), 2)
            
        elif shape == 'diamond':
            points = np.array([
                [x, y-half_size],
                [x+half_size, y],
                [x, y+half_size],
                [x-half_size, y]
            ], dtype=np.int32)
            cv2.fillPoly(image, [points], color)
            cv2.polylines(image, [points], True, (255, 255, 255), 2)
            
        elif shape == 'triangle_up':
            points = np.array([
                [x, y-half_size],
                [x-half_size, y+half_size],
                [x+half_size, y+half_size]
            ], dtype=np.int32)
            cv2.fillPoly(image, [points], color)
            cv2.polylines(image, [points], True, (255, 255, 255), 2)
            
        elif shape == 'triangle_down':
            points = np.array([
                [x, y+half_size],
                [x-half_size, y-half_size],
                [x+half_size, y-half_size]
            ], dtype=np.int32)
            cv2.fillPoly(image, [points], color)
            cv2.polylines(image, [points], True, (255, 255, 255), 2)
            
        elif shape == 'cross':
            cv2.line(image, (x-half_size, y-half_size), 
                    (x+half_size, y+half_size), color, 3)
            cv2.line(image, (x-half_size, y+half_size), 
                    (x+half_size, y-half_size), color, 3)
            cv2.circle(image, (x, y), 2, (255, 255, 255), -1)
            
    except Exception as e:
        print(f"Ошибка при рисовании маркера: {e}")

def save_centroids_to_json(centroids_data, json_path):
    """
    Сохраняет данные о центроидах в JSON файл
    """
    try:
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(centroids_data, f, ensure_ascii=False, indent=2)
        print(f"💾 Координаты центроидов сохранены в: {json_path}")
        
        # Выводим пример данных
        if centroids_data['objects']:
            sample = centroids_data['objects'][0]
            print(f"📋 Пример данных:")
            print(f"  class_id: {sample['class_id']}")
            print(f"  confidence: {sample['confidence']:.3f}")
            print(f"  GPS: ({sample['gps_coordinates']['latitude']}, {sample['gps_coordinates']['longitude']})")
            
    except Exception as e:
        print(f"❌ Ошибка при сохранении JSON: {e}")

# Упрощенная версия для быстрого использования
def process_image_simple_with_gps(model_path, image_path, output_path, conf=0.2):
    """
    Упрощенная версия без тайлинга (для небольших изображений)
    """
    model = YOLO(model_path)
    original_image = cv2.imread(image_path)
    h, w = original_image.shape[:2]
    
    # GPS координаты углов
    top_left_gps = (55.887292, 37.616198)
    bottom_right_gps = (55.915078, 37.678408)
    
    result_image = original_image.copy()
    centroids_data = {
        'metadata': {
            'image_size': {'width': w, 'height': h},
            'gps_bounds': {
                'top_left': top_left_gps,
                'bottom_right': bottom_right_gps
            },
            'processing_date': datetime.now().isoformat()
        },
        'objects': []
    }
    
    # Обрабатываем изображение
    results = model(image_path, conf=conf, imgsz=1280)
    
    for r in results:
        if r.masks is not None:
            for i, mask in enumerate(r.masks.data):
                class_id = int(r.boxes.cls[i])
                confidence = float(r.boxes.conf[i])
                
                mask_np = mask.cpu().numpy()
                mask_resized = cv2.resize(mask_np, (w, h))
                mask_binary = (mask_resized > 0.5).astype(np.uint8) * 255
                
                centroid = calculate_centroid(mask_binary)
                
                if centroid:
                    gps_coords = pixel_to_gps(centroid, w, h, top_left_gps, bottom_right_gps)
                    
                    # Сохраняем данные (только class_id, confidence и gps_coordinates)
                    centroids_data['objects'].append({
                        'class_id': class_id,
                        'confidence': confidence,
                        'gps_coordinates': {
                            'latitude': gps_coords[0], 
                            'longitude': gps_coords[1]
                        }
                    })
    
    # Сохраняем результаты
    cv2.imwrite(output_path, result_image)
    json_path = output_path.replace('.jpg', '_centroids.json').replace('.png', '_centroids.json')
    save_centroids_to_json(centroids_data, json_path)
    
    return result_image, centroids_data

# Использование
if __name__ == "__main__":
    model_path = 'runs/segment/yolov8n_gpu_simple_1/weights/best.pt'
    image_path = 'my_5km_map.jpg'
    output_path = 'my_5km_map_with_centroids.jpg'
    
    # Задаем GPS координаты углов
    top_left_gps = (55.887292, 37.616198)      # верхний левый угол
    bottom_right_gps = (55.915078, 37.678408)  # нижний правый угол
    
    print("🚀 Запуск обработки с GPS привязкой...")
    start_time = time.time()
    
    # Основная функция с тайлингом
    result, centroids_data = process_large_image_tiled_with_centroids_and_geojson(
        model_path, 
        image_path, 
        output_path,
        tile_size=640,
        overlap=64,
        conf=0.2,
        top_left_gps=top_left_gps,
        bottom_right_gps=bottom_right_gps
    )
    
    end_time = time.time()
    print(f"⏱️ Общее время обработки: {end_time - start_time:.2f} секунд")
    
    # Дополнительная информация
    print(f"\n📍 Координаты области:")
    print(f"   С-З: {top_left_gps[0]}, {top_left_gps[1]}")
    print(f"   Ю-В: {bottom_right_gps[0]}, {bottom_right_gps[1]}")

🚀 Запуск обработки с GPS привязкой...
📐 Размер исходного изображения: 11776x9472
🔲 Размер тайлов: 640x640
🔄 Перекрытие: 64 пикселей
🗺️  Географические координаты:
   Верхний левый: (55.887292, 37.616198)
   Нижний правый: (55.915078, 37.678408)
🧩 Количество тайлов: 320


Обработка тайлов: 100%|██████████| 17/17 [01:06<00:00,  3.91s/it]


💾 Изображение с центроидами сохранено: my_5km_map_with_centroids.jpg
💾 Координаты центроидов сохранены в: my_5km_map_with_centroids_centroids.json
📋 Пример данных:
  class_id: 0
  confidence: 0.681
  GPS: (55.888982, 37.616362)
✅ Обработано тайлов: 320/320
🎯 Всего обнаружено центроидов: 1664
📊 Статистика по классам:
  - building: 1521 объектов (маркер: square)
  - lake: 85 объектов (маркер: triangle_up)
  - forest: 41 объектов (маркер: circle)
  - field: 17 объектов (маркер: diamond)
⏱️ Общее время обработки: 69.17 секунд

📍 Координаты области:
   С-З: 55.887292, 37.616198
   Ю-В: 55.915078, 37.678408


In [15]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from matplotlib.colors import to_rgba
import matplotlib as mpl

def create_map_from_json(json_path, output_image_path, map_size=(12, 10), point_size=50, background_color='white'):
    """
    Создает визуальную карту с точками из JSON файла
    
    Args:
        json_path: путь к JSON файлу с координатами
        output_image_path: путь для сохранения карты
        map_size: размер карты в дюймах (width, height)
        point_size: размер точек на карте
        background_color: цвет фона карты
    """
    
    # Загружаем данные из JSON
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Извлекаем метаданные
    metadata = data['metadata']
    objects = data['objects']
    
    # Цвета для разных классов
    class_colors = {
        0: 'red',      # building
        1: 'green',    # field
        2: 'darkgreen', # forest
        3: 'blue',     # lake
        4: 'gray',     # road
        5: 'orange'    # zrail
    }
    
    # Названия классов для легенды
    class_names = {
        0: 'Building',
        1: 'Field',
        2: 'Forest', 
        3: 'Lake',
        4: 'Road',
        5: 'Railway'
    }
    
    # Собираем координаты по классам
    coordinates_by_class = {}
    for obj in objects:
        class_id = obj['class_id']
        lat = obj['gps_coordinates']['latitude']
        lon = obj['gps_coordinates']['longitude']
        confidence = obj['confidence']
        
        if class_id not in coordinates_by_class:
            coordinates_by_class[class_id] = []
        
        coordinates_by_class[class_id].append((lon, lat, confidence))
    
    # Создаем карту
    plt.style.use('default')
    fig, ax = plt.subplots(figsize=map_size, facecolor=background_color)
    
    # Устанавливаем фон
    ax.set_facecolor(background_color)
    
    # Рисуем точки для каждого класса
    for class_id, coords in coordinates_by_class.items():
        if coords:  # если есть координаты для этого класса
            lons, lats, confidences = zip(*coords)
            color = class_colors.get(class_id, 'purple')  # цвет по умолчанию
            
            # Создаем scatter plot с прозрачностью в зависимости от confidence
            alphas = [min(0.3 + conf, 0.8) for conf in confidences]  # прозрачность от confidence
            
            scatter = ax.scatter(lons, lats, 
                               c=color, 
                               s=point_size, 
                               alpha=0.7,  # базовая прозрачность
                               edgecolors='black',
                               linewidth=0.5,
                               label=class_names.get(class_id, f'Class {class_id}'))
    
    # Настройки осей
    ax.set_xlabel('Долгота (Longitude)')
    ax.set_ylabel('Широта (Latitude)')
    ax.set_title('Карта объектов по координатам GPS', pad=20)
    
    # Добавляем сетку
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    
    # Добавляем легенду
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    
    # Делаем оси равными для правильного отображения пропорций
    ax.set_aspect('equal', adjustable='datalim')
    
    # Убираем рамку
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    # Сохраняем карту
    plt.tight_layout()
    plt.savefig(output_image_path, dpi=300, bbox_inches='tight', facecolor=background_color)
    plt.close()
    
    print(f"🗺️  Карта сохранена: {output_image_path}")
    print(f"📊 Всего точек на карте: {len(objects)}")
    
    # Выводим статистику
    print("📈 Статистика по классам:")
    for class_id in sorted(coordinates_by_class.keys()):
        count = len(coordinates_by_class[class_id])
        class_name = class_names.get(class_id, f'Class {class_id}')
        print(f"  - {class_name}: {count} точек")

def create_interactive_map_from_json(json_path, output_html_path):
    """
    Создает интерактивную карту с помощью Folium
    """
    try:
        import folium
        from folium.plugins import MarkerCluster
        
        # Загружаем данные из JSON
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        objects = data['objects']
        metadata = data['metadata']
        
        # Вычисляем центр карты
        gps_bounds = metadata['gps_bounds']
        center_lat = (gps_bounds['top_left'][0] + gps_bounds['bottom_right'][0]) / 2
        center_lon = (gps_bounds['top_left'][1] + gps_bounds['bottom_right'][1]) / 2
        
        # Создаем карту
        m = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=13,
            tiles='CartoDB positron'  # светлые тайлы
        )
        
        # Цвета для маркеров
        class_colors = {
            0: 'red',      # building
            1: 'green',    # field
            2: 'darkgreen', # forest
            3: 'blue',     # lake
            4: 'gray',     # road
            5: 'orange'    # zrail
        }
        
        class_names = {
            0: 'Building',
            1: 'Field',
            2: 'Forest',
            3: 'Lake', 
            4: 'Road',
            5: 'Railway'
        }
        
        # Создаем кластеры для каждого класса
        clusters = {}
        for class_id in class_colors.keys():
            clusters[class_id] = MarkerCluster(
                name=class_names[class_id]
            ).add_to(m)
        
        # Добавляем маркеры
        for obj in objects:
            class_id = obj['class_id']
            lat = obj['gps_coordinates']['latitude']
            lon = obj['gps_coordinates']['longitude']
            confidence = obj['confidence']
            
            color = class_colors.get(class_id, 'purple')
            class_name = class_names.get(class_id, f'Class {class_id}')
            
            # Создаем popup с информацией
            popup_text = f"""
            <b>{class_name}</b><br>
            Confidence: {confidence:.3f}<br>
            Lat: {lat:.6f}<br>
            Lon: {lon:.6f}
            """
            
            folium.CircleMarker(
                location=[lat, lon],
                radius=6,
                popup=popup_text,
                color=color,
                fillColor=color,
                fillOpacity=0.7,
                weight=1
            ).add_to(clusters[class_id])
        
        # Добавляем контроль слоев
        folium.LayerControl().add_to(m)
        
        # Сохраняем карту
        m.save(output_html_path)
        print(f"🌐 Интерактивная карта сохранена: {output_html_path}")
        
    except ImportError:
        print("❌ Для интерактивной карты установите folium: pip install folium")

def create_minimalist_map_from_json(json_path, output_image_path, map_size=(16, 12)):
    """
    Создает минималистичную карту только с точками на белом фоне
    """
    
    # Загружаем данные из JSON
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    objects = data['objects']
    
    # Создаем минималистичную карту
    fig, ax = plt.subplots(figsize=map_size, facecolor='white')
    ax.set_facecolor('white')
    
    # Цвета для классов
    class_colors = {
        0: '#FF6B6B',  # красный
        1: '#51CF66',  # зеленый
        2: '#2B8C3E',  # темно-зеленый
        3: '#339AF0',  # синий
        4: '#495057',  # серый
        5: '#FF922B'   # оранжевый
    }
    
    # Собираем координаты
    for obj in objects:
        class_id = obj['class_id']
        lat = obj['gps_coordinates']['latitude']
        lon = obj['gps_coordinates']['longitude']
        
        color = class_colors.get(class_id, '#9775FA')  # фиолетовый по умолчанию
        
        ax.scatter(lon, lat, c=color, s=30, alpha=0.8, edgecolors='none')
    
    # Минималистичное оформление
    ax.set_xlabel('Longitude', fontsize=10, alpha=0.7)
    ax.set_ylabel('Latitude', fontsize=10, alpha=0.7)
    ax.set_title('Object Distribution Map', fontsize=14, pad=20, alpha=0.9)
    
    # Сетка
    ax.grid(True, alpha=0.2, linestyle='-', linewidth=0.5)
    
    # Убираем рамку
    for spine in ax.spines.values():
        spine.set_visible(False)
    
    # Сохраняем
    plt.tight_layout()
    plt.savefig(output_image_path, dpi=300, bbox_inches='tight', 
                facecolor='white', edgecolor='none',
                transparent=False)
    plt.close()
    
    print(f"🎨 Минималистичная карта сохранена: {output_image_path}")

def create_density_map_from_json(json_path, output_image_path, map_size=(14, 10)):
    """
    Создает карту плотности точек
    """
    
    # Загружаем данные из JSON
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    objects = data['objects']
    
    # Извлекаем координаты
    lons = [obj['gps_coordinates']['longitude'] for obj in objects]
    lats = [obj['gps_coordinates']['latitude'] for obj in objects]
    
    # Создаем карту плотности
    fig, ax = plt.subplots(figsize=map_size, facecolor='white')
    ax.set_facecolor('white')
    
    # Создаем hexbin plot для плотности
    hb = ax.hexbin(lons, lats, gridsize=30, cmap='Blues', alpha=0.7, mincnt=1)
    
    # Цветовая шкала
    cb = fig.colorbar(hb, ax=ax)
    cb.set_label('Плотность точек', fontsize=10)
    
    # Настройки
    ax.set_xlabel('Долгота', fontsize=10)
    ax.set_ylabel('Широта', fontsize=10)
    ax.set_title('Карта плотности объектов', fontsize=14, pad=20)
    
    # Сетка
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # Сохраняем
    plt.tight_layout()
    plt.savefig(output_image_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    
    print(f"📊 Карта плотности сохранена: {output_image_path}")

# Основная функция для использования
def main():
    json_path = 'my_5km_map_with_centroids_centroids.json'  # путь к вашему JSON файлу
    
    # Создаем разные типы карт
    
    # 1. Основная карта с точками
    create_map_from_json(
        json_path, 
        'map_points.png',
        map_size=(14, 10),
        point_size=40
    )
    
    # 2. Минималистичная карта
    create_minimalist_map_from_json(
        json_path,
        'map_minimal.png', 
        map_size=(16, 12)
    )
    
    # 3. Карта плотности
    create_density_map_from_json(
        json_path,
        'map_density.png'
    )
    
    # 4. Интерактивная карта (если установлен folium)
    create_interactive_map_from_json(
        json_path,
        'interactive_map.html'
    )

if __name__ == "__main__":
    main()

🗺️  Карта сохранена: map_points.png
📊 Всего точек на карте: 1664
📈 Статистика по классам:
  - Building: 1521 точек
  - Field: 17 точек
  - Forest: 41 точек
  - Lake: 85 точек
🎨 Минималистичная карта сохранена: map_minimal.png
📊 Карта плотности сохранена: map_density.png
❌ Для интерактивной карты установите folium: pip install folium
